# 2. Estimación de edad de personas
El problema de inferir ciertas características de una persona a través de una foto de ella puede resultar bastante dificil incluso para nosotros, como por ejemplo de qué país es, la emoción que expresa, la edad que tiene, o el género. La automatización de este proceso para que máquinas logren identificar ciertas características de una persona puede ser algo crucial para el futuro desarrollo de Inteligencia Artificial.

<img src="https://i.imgur.com/6B072GE.jpg" width="60%" height="20%" />


Trabajaremos con unos datos (imágenes) en el **objetivo** de predecir la **edad** (*target value*) de la persona presente en la imagen. Los datos corresponden a 3640 imágenes de rostros de personas extraídos de la plataforma Flickr, pero, debido a que trabajamos con redes *feed forward*, se trabajará con representaciones de alto nivel, extraídas manualmente (no-aprendibles). Para ésto necesitará descargar los datos del siguiente __[link](http://chenlab.ece.cornell.edu/people/Andy/ImagesOfGroups.html)__ en el extracto de *ageGenderClassification* o a través de la consola Unix.
```
wget http://chenlab.ece.cornell.edu/projects/ImagesOfGroups/ageGenderClassification.zip
```


**Nota:** Enunciado y la parte inicial del código de importe de datos al entorno fue extraido de una tarea para el curso de Machine Learning de la UTFSM - 2019, por lo que los creditos correspondientes de estos elementos corresponden al ayudante y/o profesor del curso.

------------------

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [ ]:
import numpy as np
def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
import scipy.io as sio
mat_file = sio.loadmat("AgeGenderClassification/eventrain.mat")
mat_file_test = sio.loadmat("AgeGenderClassification/eventest.mat")

In [ ]:
data = mat_file["trcoll"][0][0] # is "tecoll" for testing set
age_true = data[1] #target
genFeat = data[0]   # Contextual features
ffcoefs = data[3]   # Fisherface space
faceGist = data[4]  # GIST features

In [ ]:
data = mat_file_test["tecoll"][0][0] # is "tecoll" for testing set
age_true_test = data[1] #target
genFeat_test = data[0]   # Contextual features
ffcoefs_test = data[3]   # Fisherface space
faceGist_test = data[4]  # GIST features

In [ ]:
# Conjunto de prueba
X_test = np.concatenate((genFeat_test, ffcoefs_test, faceGist_test), axis = 1)
Y_test = age_true_test[:,0]

In [ ]:
# Conjunto de entrenamiento
X_train = np.concatenate((genFeat, ffcoefs, faceGist), axis = 1)
Y_train = age_true[:,0]

In [ ]:
from sklearn.model_selection import train_test_split

X_train_2, x_val, Y_train_2, y_val = train_test_split(X_train, Y_train, random_state = 11235813,
                                                      test_size = .3)


In [ ]:
# Estandarización de los atributos (whitening)
from sklearn.preprocessing import StandardScaler

std_scaler = StandardScaler().fit(X_train_2)

X_train_2_scaled = std_scaler.transform(X_train_2)
x_val_scaled = std_scaler.transform(x_val)
X_test_scaled = std_scaler.transform(X_test)




In [ ]:
sn.countplot(Y_test)

In [ ]:
sn.countplot(Y_train)

In [ ]:
np.unique(Y_train)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf_reg = RandomForestRegressor(n_estimators = 100, n_jobs=-1)

In [ ]:
%%time
rf_reg.fit(X_train_2_scaled, Y_train_2)

In [ ]:
mape_rf = mean_absolute_percentage_error(Y_test, rf_reg.predict(X_test_scaled)).round(4)
mse_rf = np.sqrt(mean_squared_error(Y_test, rf_reg.predict(X_test_scaled))).round(4)
mae_rf = mean_absolute_error(Y_test, rf_reg.predict(X_test_scaled)).round(4)

-------------
# Feed-forward NN

In [ ]:
import tensorflow as tf
import keras

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers.advanced_activations import LeakyReLU

In [ ]:
X_train_2_scaled.shape[1]

In [ ]:
base_nn = Sequential()
base_nn.add(
    Dense(30, input_dim = X_train.shape[1],
         kernel_initializer = 'glorot_uniform',
         activation = 'relu',
         name = 'input_layer')
)

base_nn.add(
    Dense(50,
         kernel_initializer = 'glorot_uniform',
         activation = 'relu',
          kernel_regularizer = 'l2',
         name = 'hidden_1')
)

base_nn.add(
    Dense(1,
         kernel_initializer = 'glorot_uniform',
         activation = 'tanh',
         name = 'out')
)


In [ ]:
base_nn.summary()

In [ ]:
from keras.callbacks import History
history = History()

In [ ]:
from keras.optimizers import Adagrad

base_nn.compile(optimizer=Adagrad(), loss = 'mean_absolute_error', metrics = ['mse'])
base_nn.fit(X_train_2_scaled, Y_train_2, epochs = 100, batch_size = 128, verbose = 0,
            callbacks=[history], validation_data=(x_val_scaled, y_val))

In [ ]:
history.history.keys()

In [ ]:
loss_train = history.history['loss']
loss_val = history.history['val_loss']
metric_train = history.history['mean_squared_error']
metric_val = history.history['val_mean_squared_error']

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sn
%matplotlib inline
print('Final loss (Validation): {0}'.format(loss_val[-1].round(4)))

fig,_ = plt.subplots(figsize = (16,8))
plt.title('MAE across training for 200 epoch', size = 20)
plt.xlabel('Epoch', size = 15)
plt.ylabel('Mean Absolute Error', size = 15)
plt.plot(range(1,101), loss_train, '-', label = 'Train MAE');
plt.plot(range(1,101), loss_val, '-', label = 'Validation MAE');
#plt.plot(range(1,101), metric_val, '-', label = 'Validation Metric: MSE');
#plt.plot(range(1,101), metric_train, '-', label = 'Train Metric: MSE');
plt.legend();
sn.despine()


In [ ]:
X_test_scaled = std_scaler.transform(X_test)

def calculate_metrics(model, x_test, y_test):
    mape = mean_absolute_percentage_error(y_test, model.predict(x_test)).round(4)
    mse= mean_squared_error(y_test, model.predict(x_test)).round(4)
    mae = mean_absolute_error(y_test, model.predict(x_test)).round(4)
    
    print('\n##################### {} ##################### '.format(type(model).__name__))
    print('MAPE: {}'.format(mape))
    print('MSE: {}'.format(mse))
    print('MAE: {}'.format(mae))
    return (mape,mse,mae)

In [ ]:
trained_models = [rf_reg, base_nn]

for model in trained_models:
    calculate_metrics(model, X_test_scaled, Y_test)

In [ ]:
np.unique(base_nn.predict(X_test))

In [ ]:
sn.distplot(base_nn.predict(X_test))

In [ ]:
sn.distplot(Y_test)

In [ ]:
np.unique(Y_test)

----------
## Otra arquitectura

In [ ]:
from keras.constraints import maxnorm

simple_nn = Sequential()

simple_nn.add(Dropout(0.2,input_shape = (X_train.shape[1],)))
simple_nn.add(
    Dense(200,
         kernel_initializer = 'glorot_uniform',
         activation = 'relu',
         name = 'input_layer')
)
simple_nn.add(Dropout(0.2))

simple_nn.add(
    Dense(500,
         kernel_initializer = 'glorot_uniform',
         kernel_constraint=maxnorm(3),
         activation = 'tanh',
         name = 'hidden_1')
)
simple_nn.add(Dropout(0.2))

simple_nn.add(
    Dense(500,
         kernel_initializer = 'glorot_uniform',
         kernel_constraint=maxnorm(3),
         activation = 'relu',
         name = 'hidden_2')
)
simple_nn.add(Dropout(0.2))

simple_nn.add(
    Dense(500,
         kernel_initializer = 'glorot_uniform',
         kernel_constraint=maxnorm(3),
         activation = 'tanh',
         name = 'hidden_3')
)
simple_nn.add(Dropout(0.2))

simple_nn.add(
    Dense(500,
         kernel_initializer = 'glorot_uniform',
         kernel_constraint=maxnorm(3),
         activation = 'tanh',
         name = 'hidden_4')
)
simple_nn.add(Dropout(0.2))

simple_nn.add(
    Dense(500,
         kernel_initializer = 'glorot_uniform',
         kernel_constraint=maxnorm(3),
         activation = 'relu',
         name = 'hidden_5')
)


simple_nn.add(
    Dense(200,
         kernel_initializer = 'glorot_uniform',
         activation = 'relu',
         name = 'hidden_6')
)
simple_nn.add(
    Dense(1, activation = 'tanh', name = 'out')
)


In [ ]:
simple_nn.summary()

In [ ]:
from keras.optimizers import Adam

history_simple_nn = History()

#sgd = SGD(lr=0.1, momentum=0.9)

simple_nn.compile(optimizer=Adam(), loss = 'mean_absolute_error')

simple_nn.fit(X_train_2_scaled, Y_train_2, epochs = 300, batch_size = 16, verbose = 1,
            callbacks=[history_simple_nn], validation_data=(x_val_scaled, y_val))

In [ ]:
loss_train_simple = history_simple_nn.history['loss']
loss_val_simple = history_simple_nn.history['val_loss']


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sn
%matplotlib inline
print('Final loss (Validation): {0}'.format(loss_val[-1].round(4)))

fig,_ = plt.subplots(figsize = (16,8))
plt.title('MAE across training for 100 epoch', size = 20)
plt.xlabel('Epoch', size = 15)
plt.ylabel('Mean Absolute Error', size = 15)
plt.plot(range(1,101), loss_train_simple, '-', label = 'Train MAE');
plt.plot(range(1,101), loss_val_simple, '-', label = 'Validation MAE');
plt.legend();
sn.despine()


In [ ]:
trained_models = [rf_reg, base_nn, simple_nn]
for model in trained_models:
    calculate_metrics(model, X_test_scaled, Y_test)

In [ ]:
sn.distplot(simple_nn.predict(X_test_scaled))

In [ ]:
np.unique(simple_nn.predict(X_test_scaled))

In [ ]:
sn.countplot(Y_test)